Epocs = 40, dataset 3000 

In [4]:
import pandas as pd
import glob
import numpy
import csv
from nltk.tokenize import word_tokenize
import xlrd
import re
from nltk import bigrams
from nltk import ngrams
import operator
from collections import Counter
from nltk.corpus import stopwords
import string
import matplotlib.pyplot as plt

In [5]:
from random import randint
from numpy import array
from numpy import argmax
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [6]:
path = r'E:\Level 4 Project\dataset\final\preprocess' # use your path
all_files = glob.glob(path + "/*.csv")

In [7]:
li = []

In [8]:
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

In [9]:
df = pd.concat(li, axis=0, ignore_index=True, sort=False)

In [10]:
df.head()

,Date,Tweet
0,2011-01-01 23:58:38+00:00,Music for space tourism Download rapidshare m...
1,2011-01-01 23:58:26+00:00,Rajasthan as Tourism Destinations http://bit....
2,2011-01-01 23:57:34+00:00,"California decriminalizes pot, under a certai..."
3,2011-01-01 23:57:28+00:00,NPL Tourism year begins with fair in Lake Cit...
4,2011-01-01 23:54:57+00:00,Though I suppose a better comparison would be...


In [11]:
df.shape

(10295057, 2)

In [12]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""

In [13]:
def preprocess_fornothash(s, lowercase=True):
    tokens = tokenize_fornothash(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

def tokenize_fornothash(s):
    return tokens_re_fornothash.findall(s)


regex_str_fornothash = [
    emoticons_str,
    r'<[^>]+>',  # HTML tags
    r'(?:@[\w_]+)',  # @-mentions
    #r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)",  # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',  # URLs

    r'(?:(?:\d+,?)+(?:\.?\d+)?)',  # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])",  # words with - and '
    r'(?:[\w_]+)',  # other words
    r'(?:\S)'  # anything else
]

tokens_re_fornothash = re.compile(r'(' + '|'.join(regex_str_fornothash) + ')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^' + emoticons_str + '$', re.VERBOSE | re.IGNORECASE)

In [14]:
def preprocess_forhash(s, lowercase=True):
    tokens = tokenize_forhash(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

def tokenize_forhash(s):
    return tokens_re_forhash.findall(s)

regex_str_forhash = [
    emoticons_str,
    r'<[^>]+>',  # HTML tags
    r'(?:@[\w_]+)',  # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)",  # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',  # URLs

    r'(?:(?:\d+,?)+(?:\.?\d+)?)',  # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])",  # words with - and '
    r'(?:[\w_]+)',  # other words
    r'(?:\S)'  # anything else
]

tokens_re_forhash = re.compile(r'(' + '|'.join(regex_str_forhash) + ')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^' + emoticons_str + '$', re.VERBOSE | re.IGNORECASE)

In [15]:
punctuation = list(string.punctuation)
unnecssary = ['The','tourism','I','rt', 'via','...','â','click']
unnecssary_hash = ['#','#tourism']
ascii_words = ['\x92','\x96']
years = ['2018','2017','2016','2015','2014','2013']
numbers = ['nine']
stop = stopwords.words('english') + punctuation + unnecssary + ascii_words + years + unnecssary_hash + numbers

count_all = []
popular_common_words = []
i=0
sets=3000

c =0
for x in range(sets):
    count_all.append(Counter())

for y in range(sets):
    for x in range(1, 500):
        value = df.loc[i, "Tweet"]

        #terms_all = [term for term in preprocess_fornothash(value) if term not in stop]  
        terms_all = [term for term in preprocess_forhash(value)
                  if term.startswith('#') if term not in stop]
        i = i + 1
        count_all[y].update(terms_all)
    
    popular_common_words.append(count_all[y].most_common(5))
        

In [16]:
trendingTopicsAll = []

def processTrendingArray(str):
    global trendingTopicsAll
    b = [i for i,x in enumerate(trendingTopicsAll) if x == str]
    if(b == []):
        trendingTopicsAll.append(str)
        return len(trendingTopicsAll)-1
    else:
        return b[0]

In [17]:
trendingTopicsInEachDays = []
trendingIdsInEachDays = []
for index, y in enumerate(popular_common_words):
    trendingTopicsInEachDays.append([])
    trendingIdsInEachDays.append([])
    for x in y:
        trendingTopicsInEachDays[index].append(x[0])
        trendingIdsInEachDays[index].append(processTrendingArray(x[0]))
        

# print(trendingTopicsInEachDays)
# print("####################")
print(trendingTopicsAll)
# print("####################")
# print(trendingIdsInEachDays)

['#travel', '#news', '#jobs', '#guyana', '#jazz', '#ecotourism', '#sgt11', '#vacation', '#holiday', '#jamaica', '#hotel', '#lodging', '#helpdot', '#gtnews', '#jo', '#amman', '#pakistan', '#nyc', '#traveltuesday', '#writer', '#bali', '#asia', '#kiva', '#africa', '#virginia', '#peru', '#savevisitlincolnshire', '#leisure', '#wine', '#wonderfulindonesia', '#qld', '#cvb', '#realestate', '#theashes', '#tourismchat', '#backpacking', '#snowfes62', '#marketing', '#freelance', '#ff', '#seo', '#abta', '#cruise', '#followfriday', '#job', '#queensland', '#adventure', '#whitsundays', '#art', '#13', '#food', '#cuisine', '#portugal', '#scholarship', '#indonesia', '#services', '#in', '#thailand', '#costarica', '#southafrica', '#australia', '#palestine', '#smbaustin', '#kerala', '#qldfloods', '#cbrt2011', '#tni', '#attraction', '#nepal', '#sapporo', '#cnn', '#tweetmyjobs', '#medical', '#masum', '#tunisia', '#canada', '#yyj', '#history', '#lp', '#jordan', '#tradeshow', '#tibet', '#tibetan', "#china's", '

In [19]:
print(trendingIdsInEachDays)

[[0, 1, 2, 3, 4], [0, 1, 3, 4, 5], [0, 2, 1, 3, 4], [0, 3, 5, 4, 1], [6, 0, 1, 7, 8], [6, 0, 1, 5, 9], [0, 10, 11, 12, 1], [1, 0, 13, 14, 15], [0, 1, 12, 13, 16], [0, 1, 12, 13, 3], [0, 17, 18, 1, 11], [0, 10, 11, 2, 19], [0, 6, 18, 20, 10], [0, 13, 1, 21, 5], [0, 12, 1, 22, 23], [0, 13, 1, 9, 24], [0, 1, 25, 10, 11], [0, 1, 26, 25, 27], [0, 1, 13, 21, 28], [13, 1, 0, 29, 30], [0, 31, 32, 1, 33], [0, 34, 1, 10, 11], [0, 10, 11, 25, 35], [1, 0, 13, 12, 29], [0, 12, 36, 1, 37], [0, 38, 2, 39, 40], [41, 0, 42, 39, 1], [10, 11, 0, 41, 1], [41, 0, 43, 13, 44], [0, 13, 43, 1, 36], [0, 6, 2, 45, 1], [0, 1, 25, 35, 16], [0, 46, 30, 47, 48], [0, 47, 30, 2, 49], [0, 1, 50, 51, 52], [0, 1, 13, 8, 47], [0, 53, 1, 28, 10], [0, 10, 11, 6, 47], [0, 10, 1, 11, 13], [0, 13, 54, 30, 55], [0, 56, 1, 57, 49], [0, 10, 1, 11, 25], [10, 11, 0, 7, 58], [0, 1, 11, 10, 13], [0, 13, 1, 23, 59], [0, 13, 12, 60, 61], [0, 47, 10, 11, 1], [0, 10, 11, 62, 13], [0, 1, 10, 11, 13], [13, 63, 64, 65, 45], [0, 13, 1, 31, 

In [34]:
# generate a sequence of random numbers in [0, 99]
tweetIdsCount = -1
inputs = len(trendingTopicsAll)

def generate_sequence():
    global tweetIdsCount
    print("tweetIdsCount - ",tweetIdsCount)
    tweetIdsCount = tweetIdsCount+1
    return (trendingIdsInEachDays[tweetIdsCount])
    #return [23,12,45]
    
# one hot encode sequence
def one_hot_encode(sequence, n_unique=inputs):
	print(sequence)    
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]   
		vector[value] = 1
		encoding.append(vector)
	print(numpy.array(encoding))
	return array(encoding)
 
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]
 
# generate data for the lstm
def generate_data():
	# generate sequence
	sequence = generate_sequence()
	# one hot encode
	print("sequence")
	encoded = one_hot_encode(sequence)
	# convert to 3d for input
	X = encoded.reshape(encoded.shape[0], 1, encoded.shape[1])
	return X, encoded    
    
# define model
model = Sequential() 
model.add(LSTM(50, input_shape=(1, inputs)))
model.add(Dense(inputs, activation='relu'))
model.add(Dense(inputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
for i in range(2000):
	X, y = generate_data()
	model.fit(X, y, epochs=40, batch_size=1, verbose=2)
# evaluate model on new data
X, y = generate_data()
yhat = model.predict(X)

#print('Expected:  %s' % one_hot_decode(y))
#print('Predicted: %s' % one_hot_decode(yhat))

a = []
b = []
i = 0;
# for x in range(5):
#     if(one_hot_decode(y)[x] == one_hot_decode(yhat)[x]):
#         i += 1
#     a.append(trendingTopicsAll[one_hot_decode(y)[x]])
#     b.append(trendingTopicsAll[one_hot_decode(yhat)[x]])
   
# print("Expected: ",a)
# print("Predicted: ",b)
# print("Test Accuracy: ",(i/5)*100)

tweetIdsCount -  -1
sequence
[0, 1, 2, 3, 4]
[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Epoch 1/40
5/5 - 2s - loss: 7.4890 - acc: 0.0000e+00
Epoch 2/40


KeyboardInterrupt: 

In [16]:
print(tweetIdsCount)
testaccuracy = []

def getOneAccuracy(a,b):
    i = 0;
    for x in range(5):
        if(a[x] == b[x]):
            i += 1
    return i/5

def getTestData():
    for i in range(990):
        X, y = generate_data()
        yhat = model.predict(X)
        print('Expected:  %s' % one_hot_decode(y))
        print('Predicted: %s' % one_hot_decode(yhat))
        testaccuracy.append(getOneAccuracy(one_hot_decode(y),one_hot_decode(yhat)))
    
getTestData()
print(testaccuracy)
print('average - ',sum(testaccuracy) / len(testaccuracy))

2000
tweetIdsCount -  2000
Expected:  [0, 1402, 311, 47, 1407]
Predicted: [0, 1402, 311, 47, 1407]
tweetIdsCount -  2001
Expected:  [0, 1402, 62, 102, 370]
Predicted: [0, 1402, 62, 102, 370]
tweetIdsCount -  2002
Expected:  [0, 132, 730, 1, 1422]
Predicted: [0, 132, 730, 1, 1420]
tweetIdsCount -  2003
Expected:  [0, 1423, 545, 547, 107]
Predicted: [0, 1420, 545, 547, 107]
tweetIdsCount -  2004
Expected:  [0, 545, 107, 730, 1418]
Predicted: [0, 545, 107, 730, 1418]
tweetIdsCount -  2005
Expected:  [0, 107, 730, 311, 780]
Predicted: [0, 107, 730, 311, 780]
tweetIdsCount -  2006
Expected:  [0, 107, 411, 545, 6]
Predicted: [0, 107, 411, 545, 6]
tweetIdsCount -  2007
Expected:  [0, 547, 545, 1424, 730]
Predicted: [0, 547, 545, 1420, 730]
tweetIdsCount -  2008
Expected:  [0, 730, 545, 273, 547]
Predicted: [0, 730, 545, 273, 547]
tweetIdsCount -  2009
Expected:  [0, 730, 36, 602, 516]
Predicted: [0, 730, 36, 602, 516]
tweetIdsCount -  2010
Expected:  [0, 730, 739, 1425, 779]
Predicted: [0, 73

Predicted: [0, 132, 730, 107, 1]
tweetIdsCount -  2094
Expected:  [0, 102, 1472, 101, 663]
Predicted: [0, 102, 1420, 101, 663]
tweetIdsCount -  2095
Expected:  [1473, 0, 132, 730, 1474]
Predicted: [1420, 0, 132, 730, 1420]
tweetIdsCount -  2096
Expected:  [1473, 0, 1472, 730, 1475]
Predicted: [1420, 0, 1420, 730, 1420]
tweetIdsCount -  2097
Expected:  [0, 730, 107, 545, 7]
Predicted: [0, 730, 107, 545, 7]
tweetIdsCount -  2098
Expected:  [0, 730, 7, 107, 134]
Predicted: [0, 730, 7, 107, 134]
tweetIdsCount -  2099
Expected:  [0, 132, 107, 730, 547]
Predicted: [0, 132, 107, 730, 547]
tweetIdsCount -  2100
Expected:  [0, 730, 132, 7, 547]
Predicted: [0, 730, 132, 7, 547]
tweetIdsCount -  2101
Expected:  [0, 730, 545, 107, 547]
Predicted: [0, 730, 545, 107, 547]
tweetIdsCount -  2102
Expected:  [0, 107, 730, 102, 545]
Predicted: [0, 107, 730, 102, 545]
tweetIdsCount -  2103
Expected:  [12, 0, 1, 730, 78]
Predicted: [12, 0, 1, 730, 78]
tweetIdsCount -  2104
Expected:  [0, 990, 1476, 78, 131

Predicted: [0, 730, 1420, 21, 22]
tweetIdsCount -  2190
Expected:  [0, 730, 1523, 1524, 1525]
Predicted: [0, 730, 1417, 1420, 1420]
tweetIdsCount -  2191
Expected:  [0, 730, 1519, 1447, 602]
Predicted: [0, 730, 1420, 1420, 602]
tweetIdsCount -  2192
Expected:  [0, 1519, 730, 27, 1]
Predicted: [0, 1420, 730, 27, 1]
tweetIdsCount -  2193
Expected:  [0, 1519, 1447, 730, 66]
Predicted: [0, 1420, 1420, 730, 66]
tweetIdsCount -  2194
Expected:  [0, 311, 1519, 814, 780]
Predicted: [0, 311, 1420, 814, 780]
tweetIdsCount -  2195
Expected:  [0, 1519, 62, 730, 1338]
Predicted: [0, 1420, 62, 730, 1420]
tweetIdsCount -  2196
Expected:  [0, 1526, 1510, 1396, 33]
Predicted: [0, 1420, 1417, 1420, 33]
tweetIdsCount -  2197
Expected:  [0, 730, 1519, 80, 50]
Predicted: [0, 730, 1420, 80, 50]
tweetIdsCount -  2198
Expected:  [0, 730, 1519, 1, 1447]
Predicted: [0, 730, 1420, 1, 1420]
tweetIdsCount -  2199
Expected:  [0, 1447, 45, 602, 1519]
Predicted: [0, 1420, 45, 602, 1420]
tweetIdsCount -  2200
Expected

Expected:  [1434, 0, 1558, 1559, 730]
Predicted: [1417, 0, 1420, 1420, 730]
tweetIdsCount -  2290
Expected:  [0, 730, 102, 50, 656]
Predicted: [0, 730, 102, 50, 1082]
tweetIdsCount -  2291
Expected:  [0, 1560, 730, 833, 134]
Predicted: [0, 1417, 730, 833, 134]
tweetIdsCount -  2292
Expected:  [0, 730, 30, 21, 794]
Predicted: [0, 730, 30, 21, 794]
tweetIdsCount -  2293
Expected:  [0, 730, 1560, 132, 921]
Predicted: [0, 730, 1417, 132, 921]
tweetIdsCount -  2294
Expected:  [0, 730, 45, 239, 833]
Predicted: [0, 730, 45, 239, 833]
tweetIdsCount -  2295
Expected:  [0, 1447, 730, 45, 219]
Predicted: [0, 1420, 730, 45, 1132]
tweetIdsCount -  2296
Expected:  [0, 643, 1, 730, 650]
Predicted: [0, 643, 1, 730, 1420]
tweetIdsCount -  2297
Expected:  [0, 730, 1519, 1561, 162]
Predicted: [0, 730, 1420, 1420, 162]
tweetIdsCount -  2298
Expected:  [0, 730, 1287, 202, 201]
Predicted: [0, 730, 1420, 202, 201]
tweetIdsCount -  2299
Expected:  [0, 602, 134, 6, 833]
Predicted: [0, 602, 134, 6, 833]
tweetId

Predicted: [0, 1420, 730, 1420, 1074]
tweetIdsCount -  2379
Expected:  [0, 730, 734, 1519, 44]
Predicted: [0, 730, 734, 1420, 44]
tweetIdsCount -  2380
Expected:  [0, 1602, 730, 1519, 1]
Predicted: [0, 1420, 730, 1420, 1]
tweetIdsCount -  2381
Expected:  [0, 1602, 730, 1603, 30]
Predicted: [0, 1420, 730, 1420, 30]
tweetIdsCount -  2382
Expected:  [0, 730, 131, 331, 27]
Predicted: [0, 730, 131, 879, 27]
tweetIdsCount -  2383
Expected:  [0, 1597, 607, 1519, 730]
Predicted: [0, 1420, 607, 1420, 730]
tweetIdsCount -  2384
Expected:  [0, 36, 45, 6, 1604]
Predicted: [0, 36, 45, 6, 1420]
tweetIdsCount -  2385
Expected:  [0, 730, 1605, 1606, 30]
Predicted: [0, 730, 1420, 1420, 30]
tweetIdsCount -  2386
Expected:  [0, 934, 730, 1519, 142]
Predicted: [0, 934, 730, 1420, 142]
tweetIdsCount -  2387
Expected:  [0, 1607, 1608, 1609, 134]
Predicted: [0, 1420, 1417, 1420, 134]
tweetIdsCount -  2388
Expected:  [0, 1522, 134, 1, 1594]
Predicted: [0, 1420, 134, 1, 1420]
tweetIdsCount -  2389
Expected:  [

Predicted: [0, 58, 1420, 1417, 1420]
tweetIdsCount -  2469
Expected:  [1519, 1615, 0, 1, 643]
Predicted: [1420, 1420, 0, 1, 643]
tweetIdsCount -  2470
Expected:  [1519, 1615, 0, 154, 730]
Predicted: [1420, 1420, 0, 201, 730]
tweetIdsCount -  2471
Expected:  [0, 730, 1519, 1615, 1642]
Predicted: [0, 730, 1420, 1420, 1420]
tweetIdsCount -  2472
Expected:  [0, 1519, 730, 1642, 1615]
Predicted: [0, 1420, 730, 1420, 1420]
tweetIdsCount -  2473
Expected:  [734, 0, 1519, 730, 1642]
Predicted: [734, 0, 1420, 730, 1420]
tweetIdsCount -  2474
Expected:  [0, 1519, 1602, 1643, 1615]
Predicted: [0, 1420, 1420, 1420, 1420]
tweetIdsCount -  2475
Expected:  [0, 1602, 1519, 1615, 1]
Predicted: [0, 1420, 1420, 1420, 1]
tweetIdsCount -  2476
Expected:  [1519, 0, 1615, 730, 1641]
Predicted: [1420, 0, 1420, 730, 1420]
tweetIdsCount -  2477
Expected:  [0, 1641, 1644, 730, 101]
Predicted: [0, 1420, 1420, 730, 101]
tweetIdsCount -  2478
Expected:  [0, 730, 1519, 921, 1599]
Predicted: [0, 730, 1420, 921, 1420]

Expected:  [0, 734, 1690, 730, 655]
Predicted: [0, 734, 1420, 730, 1420]
tweetIdsCount -  2563
Expected:  [0, 1691, 1692, 1693, 1694]
Predicted: [0, 1420, 1420, 1420, 1420]
tweetIdsCount -  2564
Expected:  [0, 1684, 730, 1695, 44]
Predicted: [0, 1420, 730, 1420, 44]
tweetIdsCount -  2565
Expected:  [0, 1696, 730, 833, 52]
Predicted: [0, 1420, 730, 833, 52]
tweetIdsCount -  2566
Expected:  [215, 0, 730, 1519, 1599]
Predicted: [1420, 0, 730, 1420, 1420]
tweetIdsCount -  2567
Expected:  [0, 215, 116, 1328, 1519]
Predicted: [0, 1420, 116, 1328, 1420]
tweetIdsCount -  2568
Expected:  [0, 1697, 45, 1519, 730]
Predicted: [0, 1420, 45, 1420, 730]
tweetIdsCount -  2569
Expected:  [0, 730, 1698, 1699, 655]
Predicted: [0, 730, 1420, 1420, 1420]
tweetIdsCount -  2570
Expected:  [0, 730, 1699, 1700, 1701]
Predicted: [0, 730, 1420, 1420, 1420]
tweetIdsCount -  2571
Expected:  [0, 1702, 405, 730, 15]
Predicted: [0, 1420, 405, 730, 15]
tweetIdsCount -  2572
Expected:  [0, 730, 713, 176, 56]
Predicted:

Expected:  [0, 730, 73, 370, 30]
Predicted: [0, 730, 73, 370, 30]
tweetIdsCount -  2743
Expected:  [0, 730, 299, 840, 102]
Predicted: [0, 730, 1390, 108, 102]
tweetIdsCount -  2744
Expected:  [0, 1790, 1791, 713, 73]
Predicted: [0, 1420, 1420, 1420, 73]
tweetIdsCount -  2745
Expected:  [0, 734, 730, 1519, 1599]
Predicted: [0, 734, 730, 1420, 1420]
tweetIdsCount -  2746
Expected:  [0, 311, 1602, 780, 730]
Predicted: [0, 311, 1420, 780, 730]
tweetIdsCount -  2747
Expected:  [0, 1602, 73, 1068, 1792]
Predicted: [0, 1420, 73, 1420, 1420]
tweetIdsCount -  2748
Expected:  [0, 730, 1, 73, 49]
Predicted: [0, 730, 1, 73, 984]
tweetIdsCount -  2749
Expected:  [0, 1519, 730, 1, 1599]
Predicted: [0, 1420, 730, 1, 1420]
tweetIdsCount -  2750
Expected:  [0, 730, 1793, 1794, 1]
Predicted: [0, 730, 1420, 1420, 1]
tweetIdsCount -  2751
Expected:  [1788, 0, 1793, 1794, 730]
Predicted: [1420, 0, 1420, 1420, 730]
tweetIdsCount -  2752
Expected:  [0, 1795, 1519, 219, 73]
Predicted: [0, 1420, 1420, 1132, 73

Expected:  [1837, 0, 73, 730, 1]
Predicted: [1420, 0, 73, 730, 1]
tweetIdsCount -  2839
Expected:  [21, 22, 134, 0, 73]
Predicted: [21, 22, 134, 0, 73]
tweetIdsCount -  2840
Expected:  [0, 64, 1838, 794, 73]
Predicted: [0, 64, 1420, 794, 73]
tweetIdsCount -  2841
Expected:  [0, 734, 36, 1839, 45]
Predicted: [0, 734, 36, 1420, 45]
tweetIdsCount -  2842
Expected:  [0, 311, 780, 73, 19]
Predicted: [0, 311, 780, 73, 19]
tweetIdsCount -  2843
Expected:  [0, 1840, 730, 73, 1]
Predicted: [0, 1420, 730, 73, 1]
tweetIdsCount -  2844
Expected:  [0, 730, 73, 80, 1519]
Predicted: [0, 730, 73, 80, 1420]
tweetIdsCount -  2845
Expected:  [0, 299, 730, 73, 1]
Predicted: [0, 1390, 730, 73, 1]
tweetIdsCount -  2846
Expected:  [0, 730, 1839, 1, 1841]
Predicted: [0, 730, 1420, 1, 1420]
tweetIdsCount -  2847
Expected:  [0, 730, 1839, 73, 142]
Predicted: [0, 730, 1420, 73, 142]
tweetIdsCount -  2848
Expected:  [0, 73, 1839, 794, 921]
Predicted: [0, 73, 1420, 794, 921]
tweetIdsCount -  2849
Expected:  [0, 73

Expected:  [734, 0, 73, 131, 102]
Predicted: [734, 0, 73, 131, 102]
tweetIdsCount -  2929
Expected:  [0, 1884, 730, 1, 30]
Predicted: [0, 1420, 730, 1, 30]
tweetIdsCount -  2930
Expected:  [0, 1519, 1599, 833, 16]
Predicted: [0, 1420, 1420, 833, 16]
tweetIdsCount -  2931
Expected:  [0, 730, 73, 1197, 1854]
Predicted: [0, 730, 73, 1417, 1420]
tweetIdsCount -  2932
Expected:  [0, 73, 299, 730, 1519]
Predicted: [0, 73, 1390, 730, 1420]
tweetIdsCount -  2933
Expected:  [1519, 0, 1599, 8, 73]
Predicted: [1420, 0, 1420, 8, 73]
tweetIdsCount -  2934
Expected:  [0, 730, 73, 1, 101]
Predicted: [0, 730, 73, 1, 101]
tweetIdsCount -  2935
Expected:  [0, 1885, 73, 266, 1886]
Predicted: [0, 1420, 73, 1420, 1420]
tweetIdsCount -  2936
Expected:  [0, 1886, 266, 1887, 730]
Predicted: [0, 1420, 1420, 1420, 730]
tweetIdsCount -  2937
Expected:  [0, 16, 73, 118, 1888]
Predicted: [0, 16, 73, 118, 1420]
tweetIdsCount -  2938
Expected:  [0, 73, 52, 1889, 132]
Predicted: [0, 73, 52, 1420, 132]
tweetIdsCount -

In [17]:
len(trendingTopicsAll)

1934

In [19]:
from tensorflow.keras.models import model_from_json

In [20]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [21]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

W0117 23:59:24.137207 15184 deprecation.py:506] From C:\Users\Mithila\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0117 23:59:24.143220 15184 deprecation.py:506] From C:\Users\Mithila\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling Orthogonal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0117 23:59:24.147231 15184 deprecation.py:506] From C:\Users\Mithila\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and

Loaded model from disk


In [22]:
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
X, y = generate_data()
yhat = loaded_model.predict(X)
print('Expected:  %s' % one_hot_decode(y))
print('Predicted: %s' % one_hot_decode(yhat))

tweetIdsCount -  2990
Expected:  [0, 730, 73, 118, 81]
Predicted: [0, 730, 73, 118, 81]


In [24]:
def getTestData():
    for i in range(9):
        X, y = generate_data()
        yhat = loaded_model.predict(X)
        print('Expected:  %s' % one_hot_decode(y))
        print('Predicted: %s' % one_hot_decode(yhat))
        testaccuracy.append(getOneAccuracy(one_hot_decode(y),one_hot_decode(yhat)))

In [25]:
getTestData()
print(testaccuracy)

tweetIdsCount -  2991
Expected:  [0, 1929, 730, 73, 1930]
Predicted: [0, 1420, 730, 73, 1420]
tweetIdsCount -  2992
Expected:  [0, 1931, 730, 73, 602]
Predicted: [0, 1420, 730, 73, 602]
tweetIdsCount -  2993
Expected:  [0, 730, 73, 1932, 1933]
Predicted: [0, 730, 73, 1420, 1420]
tweetIdsCount -  2994
Expected:  [0, 730, 36, 537, 95]
Predicted: [0, 730, 36, 1399, 95]
tweetIdsCount -  2995
Expected:  [0, 73, 730, 1447, 794]
Predicted: [0, 73, 730, 1420, 794]
tweetIdsCount -  2996
Expected:  [0, 1402, 730, 73, 1]
Predicted: [0, 1402, 730, 73, 1]
tweetIdsCount -  2997
Expected:  [0, 118, 494, 730, 73]
Predicted: [0, 118, 494, 730, 73]
tweetIdsCount -  2998
Expected:  [0, 73, 730, 134, 833]
Predicted: [0, 73, 730, 134, 833]
tweetIdsCount -  2999


IndexError: list index out of range